In [1]:
from __future__ import print_function

import tensorflow as tf
import numpy as np
import os

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import shutil
import pandas as pd

from IPython.display import HTML, display

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

%matplotlib inline

class Context:
    """ 
    Execution Context
    """ 
    def __init__(self, model, version=1):
        self.home = '/home/ml'
        self.model = model
        self.version = version
        self.log_path = "%s/%s/%s-%s" % (self.home, "logs", model, version)
        self.keras_path = "%s/%s/%s-%s" % (self.home, "models-keras", model, version)        
        self.model_path = "%s/%s/%s/%s" % (self.home, "models", model, version)
        self.checkpoint_path = "%s/%s/%s/%s" % (self.home, "checkpoints", model, version)

        shutil.rmtree(self.keras_path, ignore_errors=True)
        shutil.rmtree(self.model_path, ignore_errors=True)
        shutil.rmtree(self.log_path, ignore_errors=True)
        shutil.rmtree(self.checkpoint_path, ignore_errors=True)

        os.makedirs(self.log_path)
        os.makedirs(self.checkpoint_path)
    
    def __str__(self):
        return """
    Context {
        model: %s,
        version: %s,
        log_path: %s,
        keras_path: %s,
        model_path: %s,
        checkpoint_path: %s
    }""" % (self.model, self.version, self.log_path, self.keras_path, self.model_path, self.checkpoint_path)
    

class _DataWrapper:
    """ 
    Wrapper for data collections with a convenient next_batch method for 
    accessing the data circularily 
    """
    def __init__(self, features, labels, mask):
        self.labels = labels[mask]
        self.features = features[mask]
        self._from, self._to, self.total  = 0, 0, self.features.shape[0]
        
    def next_batch(self, batch_size):
        if self._to == self.total:
            self._from = 0
            
        self._to = min(self._from + batch_size, self.total)
        s = slice(self._from, self._to)
        
        return self.features[s], self.labels[s]
    
    def head(self):
        return self.features[1:5], self.labels[1:5]
    
    def _all(self):
        return self.features, self.labels
        

Using TensorFlow backend.
